## <b>M4_텍스트 벡터화(Text_Vectorization)</b>

- 자연어 처리에서는 기계가 문자를 이해 할 수 있도록 수치화해주는 과정이 반드시 필요하다.
- 텍스트 벡터화의 대표적인 방법 : 원-핫 인코딩(One-hot encoding), TF-IDF(빈도수 기반 텍스트/문서 벡터화), 단어 임베딩(Word Embedding)

### 1. 원-핫 인코딩(One-hot encoding)

- 인코딩 : 컴퓨터는 텍스트를 직접 처리하는게 아니라 숫자로 변환하여 처리한다.
   - 인코딩에는 텍스트를 정수로 변환하는 `정수 인코딩`과 원핫 벡터로 표현하는 `원핫 인코딩`이 있다.

- 정수 인코딩(Integer Encoding) : 자연어를 컴퓨터가 이해할 수 있는 숫자(정수) 형태로 인코딩하는 과정이다.

- 원-핫 인코딩 : 정수로 표현되었지만 실제로는 문자인 데이터를 기계가 인식할 수 있도록 버꿔주는 방법이다.
   - N개의 단어를 각각 N차원의 벡터로 표현하는 방식이다.
   - 단어에 해당되는 차원(인덱스)에 1을 넣고 나머지에는 0을 입력한다.
   - 원-핫 인코딩은 단어 또는 문자를 기준으로 벡터화가 가능하다.

- 정수 인코딩과 원-핫 인코딩(하나의 문장)

  <img src='images/encoding1.png' width='980px'>

> 정수 인코딩

In [ ]:
tokens = ['나', '는', '자연어', '처리', '를', '배운다']

- 형태소별 인덱스로 번호 부여

In [ ]:
word_to_index = {word : index for index, word in enumerate(tokens)}

print('정수 인코딩 :',word_to_index)

> 원-핫 인코딩

>> 원-핫 벡터 생성 함수 정의

In [ ]:
def one_hot_encoding(word, word_to_index):
  one_hot_vector = [0]*(len(word_to_index))
  index = word_to_index[word]
  one_hot_vector[index] = 1
  return one_hot_vector

>> 원-핫 인코딩 : 형태소별 원-핫 벡터 출력

In [ ]:
for word in tokens:
    print(word,"\t",one_hot_encoding(word, word_to_index))

### 2. TF-IDF(빈도수 기반 텍스트/문서 벡터화)

> BoW(Bag of Words)

- BoW : 등장하는 단어들의 숫자를 세서 단어 주머니에 넣어 두는 것이다.

   <img src='images/bow.jpg' width='750px'>
      
   ###### * 출처 : https://web.stanford.edu/~jurafsky/slp3/4.pdf

- BoW를 만드는 과정
   - 각 단어에 고유한 인덱스를 부여한다.
   - 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다.

>> 데이터 입력

In [ ]:
corpus = ['If you do not walk today. you will have to run tomorrow']

>> 빈도 측정

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer()

>> 불용어 제거

>> BoW

> 문서 단어 행렬(Document-Term Matrix, DTM)

- 문서 단어 행렬(Document-Term Matrix)은 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것을 말한다.
- 각 문서에 대한 BoW를 하나의 행렬로 표현한 것이다.

   <img src='images/dtm.png' width='980px'>

>> 데이터 입력

In [ ]:
docs = [
  '사고 싶은 스마트폰',
  '사고 싶은 스마트워치',
  '성능 좋은 스마트폰 스마트폰',
  '나는 스마트폰이 좋아요']

>> BoW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vector = CountVectorizer()
bow = vector.fit_transform(docs)

print(bow.toarray())
print(vector.vocabulary_)

>> DTM

In [ ]:
import pandas as pd

columns = []
for k, v in sorted(vector.vocabulary_.items(), key=lambda item:item[1]):
      columns.append(k)

df = pd.DataFrame(bow.toarray(), columns=columns)
df

> 어휘 빈도-문서 역빈도(Term Frequency-Inverse Docunment Frequency, TF-IDF) 분석

- TF-IDF : 단어의 빈도와 역 문서 빈도를 사용하여 DTM 내의 각 단어들 마다 중요한 정도를 가중치로 부여하는 것이다.
   - 단순히 빈도수가 높은 단어가 핵심어가 아닌, 특정 문서에서만 집중적으로 등장할 때 해당 단어가 문서의 주제를 잘 담고 있는 핵심어라고 가정한다.
   - 특정 문서에서 특정단어가 많이 등장하고 그 단어가 다른 문서에서 적게 등장할 때, 그 단어를 특정 문서의 핵심어로 간주한다.<br>
   → 특정 문서에서 특정 단어가 많이 등장하는 것을 의미한다.
   - 문서의 유사도 측정, 검색 시스템에서 검색 결과의 중요도 계산, 문서 내 특정 단어의 중요도 계산에 활용한다.
   - 어휘 빈도-문서 역빈도(tf-idf) : 어휘 빈도(tf)와 역문서 빈도(idf)를 곱해서 계산  $$ tf_{x,y} $$
   - 역문서 빈도(idf) : 다른 문서에서 등장하지 않는 단어 빈도를 의미  $$ log(N/df_x) $$      
   - 어휘 빈도-문서 역빈도(tf-idf) : 다음과 같이 표현  $$ W_{x,y} = tf_{x,y} * log(N/df_x) $$

- tf-idf 계산 방법

   <img src='images/tf-idf1.png' width='980px'>
   <img src='images/tf.png' width='980px'>
   <img src='images/idf.png' width='980px'>
   <img src='images/tf-idf2.png' width='980px'>

- tf-idf를 편리하게 계산하기 위해 `scikit-learn`의 `tfidfvectorizer`를 이용한다.
- 앞서 계산한 단어 빈도 수를 입력하여 tf-idf로 변환한다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

### 3. 단어 임베딩(Word Embedding)

- 의미를 포함하는 단어를 벡터로 바꾸는 기법이며, 비슷한 분포를 가진 단어의 주변 단어들도 비슷한 의미를 가진다는 것을 가정한다.

- 예시)
   - 왼쪽 그림을 보면 왕과 여왕, 여왕과 여자가 같은 방향에 있다.
   - 의미가 비슷한 단어는 비슷한 방향에 위치하게 된다.
   - 단어 임베딩은 단어의 의미를 효과적으로 표현하기 때문에 one-hot encoding보다 학습 성능을 높일 수 있다.
   - 대량의 데이터로 단어 임베딩을 미리 학습시켜 두면, 문서 분류와 같은 과제에서 더 적은 데이터로도 학습된 임베딩을 사용하여 높은 성능을 낼 수 있다.

- Word Embedding
   
   <img src='images/w_embed1.png' width='980px'>

   ###### * 출처 : http://doc.mindscale.kr/km/unstructured/11.html

- 희소표현 : one-hot encoding은 단어의 의미를 전혀 고려하지 않으며 벡터의 길이가 총 단어 수가 되므로 매우 희박(sparse)한 형태가 된다.
- 밀집표현 : 이를 해결하기 위해 단어의 의미를 고려하여 좀 더 조밀한 차원에 단어를 벡터로 표현하는 것을 단어 임베딩(word embedding)이라 한다.
- 원핫 벡터로 표현된 단어를 밀집 벡터(dense vector)로 변환하는것을 워드 임베딩(word embedding)이라고 한다.
   - 이렇게 만들어진 단어 벡터는 단어의 의미 담고있으며, 단어 벡터 간의 연산도 가능하다.
   - 많은 양의 문서를 학습하여 얻어진 단어 벡터는 단어 간의 관계를 보다 정확하게 나타낸다.

- 10,000개의 단어로 이루어진 단어사전에서 희소표현과 밀집표현 비교

   <img src='images/w_embed2.png' width='980px'>

- Word2vec
   - 단어를 벡터로 임베딩하는 방식은 머신러닝을 통해 학습되는데, 신경망을 기반으로 한 단어 벡터화의 대표적 방법은 Word2Vec이다.
   - Word2vec은 단어 벡터 간 유의미한 유사도를 반영할 수 있도록 단어의 의미를 수치화 하는 방법이다.(문맥기반 학습)

- Word2vec 학습 방법 : CBOW(continuous bag of words)와 Skip-gram(SG)의 두 가지 알고리즘이 있고 일반적이다.
   
   <img src='images/word2vec1.png' width='980px'><br>

> Word2Vec로 임베딩

- 형태소로 분석된 명사를 파일로 읽기

In [ ]:
f = open('data_set/risk_noun.txt', 'r', encoding='utf-8')
raw = f.read()
print(raw[:1000])

- 문자열을 리스트로 변환

> Word2Vec 훈련시키기 : Word2Vec 모델 활용

In [ ]:
from gensim.models import Word2Vec

> 유사도가 높은 단어 추출

> 저장된 학습모델로 유사도 높은 단어 추출

In [ ]:
model.wv.save_word2vec_format('word2')

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
load_model = KeyedVectors.load_word2vec_format('word2')
load_model.most_similar('마케팅', topn=5)